## Reformatting json data as csv


In [3]:
import os, csv, json, glob
import pandas as pd

# let's start by listing the .book files
files = glob.glob('ryan/*.book')
print(files)

['ryan/inu.30000003130071.book', 'ryan/mdp.39015002153180.book', 'ryan/mdp.39015002260167.book', 'ryan/mdp.39015008102165.book', 'ryan/mdp.39015008821251.book', 'ryan/mdp.39015028400326.book', 'ryan/mdp.39015029851667.book', 'ryan/mdp.39015040081708.book', 'ryan/mdp.39015040597794.book', 'ryan/mdp.39015043002289.book', 'ryan/mdp.39015046349141.book', 'ryan/mdp.39015054067759.book', 'ryan/mdp.39015054102655.book', 'ryan/mdp.39015062063501.book', 'ryan/mdp.39015063339868.book', 'ryan/pst.000018252354.book', 'ryan/pst.000018729719.book', 'ryan/pst.000022105387.book', 'ryan/pst.000044192167.book', 'ryan/uc1.$b322793.book', 'ryan/uc1.$b784307.book', 'ryan/uc1.32106002122734.book', 'ryan/uc1.32106010927215.book', 'ryan/uc1.32106011264659.book', 'ryan/uc1.32106014098278.book', 'ryan/uc1.b3462870.book', 'ryan/uc1.b4439246.book']


In [15]:
chardict = dict()
chardict['bookid'] = []
chardict['name'] = []
chardict['words'] = []

for afile in files:
    with open(afile, encoding = 'utf-8') as f:
        filestring = f.read()
        jsonobj = json.loads(filestring)
        the_id = jsonobj['id']
        allnames = set()
        for c in jsonobj['characters']:
            names = c['names']
            # let's take the name that occurs most often
            the_name = ''
            maxcount = 0
            for n in names:
                ct = n['c']
                if ct > maxcount and n['n'] not in allnames:
                    maxcount = ct
                    the_name = n['n']
            words = []
            for gram_cat in ['agent', 'patient', 'mod', 'poss']:
                for word in c[gram_cat]:
                    words.append(word['w'])
            
            if len(words) > 0:
                chardict['bookid'].append(the_id)
                chardict['name'].append(the_name)
                allnames.add(the_name)
                chardict['words'].append(' '.join(words))

post22 = pd.DataFrame(chardict)
post22.head()
            

,bookid,name,words
0,inu.30000003130071,Miss Bentley,ask ranks looked came come blushed say come tu...
1,inu.30000003130071,John Thomas,some252 comes killed getting puts say says tea...
2,inu.30000003130071,Nero,slaves slaves slaves
3,inu.30000003130071,Herbert,laughed dead trees
4,inu.30000003130071,Daniele,wear hireling looks has said betrayed leaving ...


In [17]:
# basically the same approach but just a different i/o process

chardict = dict()
chardict['bookid'] = []
chardict['name'] = []
chardict['words'] = []

with open('ryan/earlychars.jsonl', encoding = 'utf-8') as f:
    for line in f:
        jsonobj = json.loads(line)
        
        # from here in, it's the same; if I were being careful I'd
        # encapuslate it as a function to avoid repeating code, but
        # working quickly I just copy and paste
        
        the_id = jsonobj['id']
        allnames = set()
        for c in jsonobj['characters']:
            names = c['names']
            # let's take the name that occurs most often
            the_name = ''
            maxcount = 0
            for n in names:
                ct = n['c']
                if ct > maxcount and n['n'] not in allnames:
                    maxcount = ct
                    the_name = n['n']
            words = []
            for gram_cat in ['agent', 'patient', 'mod', 'poss']:
                for word in c[gram_cat]:
                    words.append(word['w'])
            
            if len(words) > 0:
                chardict['bookid'].append(the_id)
                chardict['name'].append(the_name)
                allnames.add(the_name)
                chardict['words'].append(' '.join(words))

pre23 = pd.DataFrame(chardict)
pre23.head()
        

,bookid,name,words
0,10117,Theodore,remember
1,10117,Mademoiselle,repeat pour
2,10117,Mr. Rochester,take commissioned intends does was lay carried...
3,10117,Mr. Broeklehurst,said
4,10117,Miss Temple,received name face side ear eye bed


In [18]:
allchars = pd.concat([pre23, post22])
print(pre23.shape)
print(post22.shape)
allchars.shape


(399, 3)
(2094, 3)


(2493, 3)

In [19]:
allchars.to_csv('ryan/allchars.csv')